# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [122]:
! python --version

Python 3.6.3


In [123]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql import Window

from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql import functions as F

import datetime

In [124]:
# create a Spark session
spark = SparkSession.builder \
    .master("local") \
    .appName("Sparkify") \
    .getOrCreate()

In [125]:
spark.sparkContext.getConf().getAll()

[('spark.master', 'local'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.name', 'Sparkify'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.app.id', 'local-1572026364142'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.port', '43293'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.host', 'a1a693689ad3')]

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [126]:
# load data from the json file
sparkify_data_file = "./mini_sparkify_event_data.json"
event_data = spark.read.json(sparkify_data_file)
event_data.persist()

DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: bigint, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string]

In [127]:
event_data.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

In [128]:
event_data.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [129]:
event_data.count()

286500

In [130]:
event_data.select("userId").dropDuplicates().sort("userId").show(3)

+------+
|userId|
+------+
|      |
|    10|
|   100|
+------+
only showing top 3 rows



In [131]:
event_data.filter(event_data["userId"] == "").count()

8346

In [132]:
event_data.filter(event_data["sessionId"] == "").count()

0

In [133]:
# As suggested, let's remove any row with missing data in the userId and sessionId columns
# seems useless with the subset "mini_sparkify", but most likely be useful for the large dataset
user_log_valid = event_data.dropna(how = "any", subset = ["userId", "sessionId"])

In [134]:
user_log_valid.count()

286500

In [135]:
user_log_valid.filter(user_log_valid["userId"] == "").count()

8346

In [136]:
user_log_valid = user_log_valid.filter(user_log_valid["userId"] != "")

In [137]:
user_log_valid.count()

278154

In [138]:
286500-278154

8346

In [139]:
# display function to help loop over columns
def display_column(col_name, nb_row=5):
    user_log_valid.select(col_name).distinct().show(nb_row)

In [140]:
for col in user_log_valid.schema.names: display_column(col)

+----------------+
|          artist|
+----------------+
|  The Black Keys|
|     Silverstein|
|       Kate Nash|
|    Yann Tiersen|
|Jane's Addiction|
+----------------+
only showing top 5 rows

+---------+
|     auth|
+---------+
|Cancelled|
|Logged In|
+---------+

+---------+
|firstName|
+---------+
|   Maddox|
|   Karter|
|    Lucas|
|    Grace|
| Antonina|
+---------+
only showing top 5 rows

+------+
|gender|
+------+
|     F|
|     M|
+------+

+-------------+
|itemInSession|
+-------------+
|           26|
|           29|
|          474|
|          964|
|           65|
+-------------+
only showing top 5 rows

+--------+
|lastName|
+--------+
|Harrison|
|Thornton|
|    Pena|
|   Jones|
|  Santos|
+--------+
only showing top 5 rows

+---------+
|   length|
+---------+
|200.75057|
|317.30893|
|260.28363|
|241.94567|
|262.79138|
+---------+
only showing top 5 rows

+-----+
|level|
+-----+
| free|
| paid|
+-----+

+--------------------+
|            location|
+--------------------+

So far I understand the following:

Each line of the spark dataframe is a record which is related to a user requesting a page with a PUT or GET method.
User is repsented by userId and each session has an id as well. There is even a itemInSession column that provide the sequence of all pages the users visited within a session. ts column provide a timestamp of the "event" in those logs.
UserAgent tells us about the operating system and browser used. Status is the return code of the server. Level is telling us if the user is using the free version or if he is under paid contract.
Other field are pretty obvious and their name is pretty explanatory.
The only one I don't understand is about the lenght. Let's explore a little bit this one.

In [141]:
for page in user_log_valid.select("page").distinct().collect():
    print(user_log_valid.select("page","length").where(user_log_valid.page == page[0]).distinct().collect())

[Row(page='Cancel', length=None)]
[Row(page='Submit Downgrade', length=None)]
[Row(page='Thumbs Down', length=None)]
[Row(page='Home', length=None)]
[Row(page='Downgrade', length=None)]
[Row(page='Roll Advert', length=None)]
[Row(page='Logout', length=None)]
[Row(page='Save Settings', length=None)]
[Row(page='Cancellation Confirmation', length=None)]
[Row(page='About', length=None)]
[Row(page='Settings', length=None)]
[Row(page='Add to Playlist', length=None)]
[Row(page='Add Friend', length=None)]
[Row(page='NextSong', length=515.21261), Row(page='NextSong', length=268.5122), Row(page='NextSong', length=201.79546), Row(page='NextSong', length=141.47873), Row(page='NextSong', length=259.68281), Row(page='NextSong', length=181.75955), Row(page='NextSong', length=273.42322), Row(page='NextSong', length=304.43057), Row(page='NextSong', length=185.41669), Row(page='NextSong', length=333.94893), Row(page='NextSong', length=171.33669), Row(page='NextSong', length=485.85098), Row(page='NextSon

In [142]:
user_log_valid\
.select("sessionId", "itemInSession", "ts", "page", "length", "song")\
.where((user_log_valid.page == "NextSong"))\
.sort("length")\
.show(15)

+---------+-------------+-------------+--------+--------+--------------------+
|sessionId|itemInSession|           ts|    page|  length|                song|
+---------+-------------+-------------+--------+--------+--------------------+
|     1599|           47|1542579601000|NextSong| 0.78322|La Danza De Las S...|
|      294|          263|1541135356000|NextSong| 2.16771|You Can't Fix Stupid|
|      609|           45|1539203952000|NextSong| 4.04853|               Solar|
|     2166|          320|1542989477000|NextSong| 4.04853|               Solar|
|      112|           62|1539266340000|NextSong|  4.8322|               Intro|
|     1871|           58|1542090926000|NextSong|  6.3473|              Kecy 2|
|     1188|           48|1540702723000|NextSong| 7.83628|              Chalwa|
|     1288|           47|1540528480000|NextSong| 8.30649|We're Rolling [Ba...|
|      316|          134|1538727598000|NextSong| 9.01179|         The Mistake|
|     2264|           63|1543404358000|NextSong| 9.0

Now, we understand that the length column is used only for the page NextSong and correspond to the number of second of the song.

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [143]:
# define churn looking at 'cancellation confirmation' event

In [144]:
user_log_valid \
.select("sessionId", "itemInSession")\
.groupby("sessionId")\
.agg({'itemInSession':'max'})\
.sort(F.desc('max(itemInSession)'))\
.show(3)

user_log_valid\
.where(user_log_valid["page"] == "NextSong")\
.groupby("userId")\
.agg({"page" : "count"})\
.withColumnRenamed("count(page)", "total_songs_played")\
.sort(F.desc('total_songs_played'))\
.show(3)

user_log_valid\
.where(user_log_valid["page"] == "NextSong")\
.groupby("userId")\
.agg({"length" : "sum"})\
.withColumnRenamed("sum(length)", "total_seconds_played")\
.sort(F.desc('total_seconds_played'))\
.show(3)

user_log_valid\
.groupby("userId")\
.agg(F.countDistinct("sessionId"))\
.withColumnRenamed("count(DISTINCT sessionId)", "nb_sessions")\
.sort(F.desc('nb_sessions'))\
.show(3)

+---------+------------------+
|sessionId|max(itemInSession)|
+---------+------------------+
|     1840|              1321|
|      316|              1099|
|      635|               889|
+---------+------------------+
only showing top 3 rows

+------+------------------+
|userId|total_songs_played|
+------+------------------+
|    39|              8002|
|    92|              5945|
|   140|              5664|
+------+------------------+
only showing top 3 rows

+------+--------------------+
|userId|total_seconds_played|
+------+--------------------+
|    39|  1991154.9155799907|
|    92|   1491493.286370003|
|   140|  1410075.4463299962|
+------+--------------------+
only showing top 3 rows

+------+-----------+
|userId|nb_sessions|
+------+-----------+
|    39|        107|
|    92|         86|
|200023|         76|
+------+-----------+
only showing top 3 rows



In [145]:
user_log_valid\
.groupby("userAgent")\
.agg(F.countDistinct("userId"))\
.withColumnRenamed("count(DISTINCT userId)", "nb_users")\
.sort(F.desc('nb_users'))\
.collect()[0]

Row(userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', nb_users=24)

In [155]:
# we need to find the string between the first parenthesis and the first semi colon
import re
os_udf = udf(lambda x: re.findall('\(([^\);]*)[;|\)]', x)[0] , StringType())
# we had a column with the OS
user_log_valid = user_log_valid.withColumn("OS", os_udf(user_log_valid.userAgent))

In [156]:
user_log_valid.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30', OS='Windows NT 6.1')

In [158]:
user_log_valid\
.groupBy("OS")\
.agg({"OS":"count"})\
.collect()

[Row(OS='iPad', count(OS)=11381),
 Row(OS='X11', count(OS)=8961),
 Row(OS='Windows NT 6.3', count(OS)=18530),
 Row(OS='iPhone', count(OS)=11192),
 Row(OS='Windows NT 6.2', count(OS)=5427),
 Row(OS='Macintosh', count(OS)=102749),
 Row(OS='Windows NT 5.1', count(OS)=7406),
 Row(OS='compatible', count(OS)=10684),
 Row(OS='Windows NT 6.0', count(OS)=2442),
 Row(OS='Windows NT 6.1', count(OS)=99382)]

In [ ]:
# add an OS column (get that info from the userAgent text)


In [18]:
user_log_valid \
.select(["sessionId", "userAgent"]) \
.where(user_log_valid.userId == '18') \
.collect()

[Row(sessionId=242, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4"'),
 Row(sessionId=242, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4"'),
 Row(sessionId=242, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4"'),
 Row(sessionId=242, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4"'),
 Row(sessionId=242, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4"'),
 Row(sessionId=242, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4"'),
 Row(sessionId=242, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_

In [55]:
user_log_valid \
.select(["userId", "sessionId", "firstname", "page", "song"]) \
.where(user_log_valid.page == "Cancellation Confirmation") \
.collect()

[Row(userId='18', sessionId=514, firstname='Adriel', page='Cancellation Confirmation', song=None),
 Row(userId='32', sessionId=540, firstname='Diego', page='Cancellation Confirmation', song=None),
 Row(userId='125', sessionId=174, firstname='Mason', page='Cancellation Confirmation', song=None),
 Row(userId='105', sessionId=508, firstname='Alexander', page='Cancellation Confirmation', song=None),
 Row(userId='17', sessionId=797, firstname='Kayla', page='Cancellation Confirmation', song=None),
 Row(userId='143', sessionId=843, firstname='Molly', page='Cancellation Confirmation', song=None),
 Row(userId='101', sessionId=842, firstname='Alex', page='Cancellation Confirmation', song=None),
 Row(userId='129', sessionId=802, firstname='Davis', page='Cancellation Confirmation', song=None),
 Row(userId='121', sessionId=881, firstname='Nikolas', page='Cancellation Confirmation', song=None),
 Row(userId='51', sessionId=934, firstname='Ethan', page='Cancellation Confirmation', song=None),
 Row(use

In [21]:
user_log_valid \
.select(["userId", "sessionId", "firstname", "page", "song"]) \
.where(user_log_valid.page == "Submit Downgrade") \
.collect()

[Row(userId='131', sessionId=249, firstname='Kael', page='Submit Downgrade', song=None),
 Row(userId='38', sessionId=313, firstname='Calvin', page='Submit Downgrade', song=None),
 Row(userId='141', sessionId=479, firstname='Kaylenn', page='Submit Downgrade', song=None),
 Row(userId='30', sessionId=532, firstname='Colin', page='Submit Downgrade', song=None),
 Row(userId='20', sessionId=378, firstname='Destiny', page='Submit Downgrade', song=None),
 Row(userId='12', sessionId=632, firstname='Sophia', page='Submit Downgrade', song=None),
 Row(userId='11', sessionId=487, firstname='Zoe', page='Submit Downgrade', song=None),
 Row(userId='61', sessionId=529, firstname='Parker', page='Submit Downgrade', song=None),
 Row(userId='81', sessionId=725, firstname='Oliver', page='Submit Downgrade', song=None),
 Row(userId='85', sessionId=734, firstname='Brayden', page='Submit Downgrade', song=None),
 Row(userId='140', sessionId=753, firstname='Lucero', page='Submit Downgrade', song=None),
 Row(userI

In [22]:
flag_downgrade_event = udf(lambda x: 1 if x == "Submit Downgrade" else 0, IntegerType())
flag_cancellation_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())

In [23]:
user_log_valid = user_log_valid.withColumn("downgrade", flag_downgrade_event("page"))
user_log_valid = user_log_valid.withColumn("churn", flag_cancellation_event("page"))

In [24]:
user_log_valid.head(1)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30', downgrade=0, churn=0)]

In [25]:
user_log_valid \
.select(["userId", "ts", "page", "level", "churn", "downgrade"]) \
.filter(user_log_valid.userId == "18").filter(user_log_valid.churn == "1") \
.sort("ts") \
.collect()

[Row(userId='18', ts=1538943990000, page='Cancellation Confirmation', level='paid', churn=1, downgrade=0)]

In [26]:
user_log_valid \
.select(["userId", "ts", "page", "level", "churn", "downgrade"]) \
.filter(user_log_valid.downgrade == "1") \
.sort("ts") \
.show(user_log_valid.count(),False)

+------+-------------+----------------+-----+-----+---------+
|userId|ts           |page            |level|churn|downgrade|
+------+-------------+----------------+-----+-----+---------+
|131   |1538393619000|Submit Downgrade|paid |0    |1        |
|38    |1538516445000|Submit Downgrade|paid |0    |1        |
|200003|1538744281000|Submit Downgrade|paid |0    |1        |
|200023|1538753821000|Submit Downgrade|paid |0    |1        |
|141   |1538835479000|Submit Downgrade|paid |0    |1        |
|30    |1538995454000|Submit Downgrade|paid |0    |1        |
|20    |1538997356000|Submit Downgrade|paid |0    |1        |
|300015|1539027805000|Submit Downgrade|paid |0    |1        |
|100018|1539106390000|Submit Downgrade|paid |0    |1        |
|12    |1539144047000|Submit Downgrade|paid |0    |1        |
|11    |1539239742000|Submit Downgrade|paid |0    |1        |
|61    |1539244188000|Submit Downgrade|paid |0    |1        |
|81    |1539264337000|Submit Downgrade|paid |0    |1        |
|200025|

In [27]:
# find the users who have churn and downgraded


In [28]:
downgrade_logs = user_log_valid.filter(user_log_valid.downgrade == "1")
downgrade_logs.select(['userID','page', 'firstName', 'lastName','ts', 'auth']).show(5, False)

+------+----------------+---------+--------+-------------+---------+
|userID|page            |firstName|lastName|ts           |auth     |
+------+----------------+---------+--------+-------------+---------+
|131   |Submit Downgrade|Kael     |Baker   |1538393619000|Logged In|
|38    |Submit Downgrade|Calvin   |Marshall|1538516445000|Logged In|
|141   |Submit Downgrade|Kaylenn  |Jenkins |1538835479000|Logged In|
|30    |Submit Downgrade|Colin    |Freeman |1538995454000|Logged In|
|20    |Submit Downgrade|Destiny  |Herring |1538997356000|Logged In|
+------+----------------+---------+--------+-------------+---------+
only showing top 5 rows



In [29]:
churn_logs = user_log_valid.filter(user_log_valid.churn == "1")
churn_logs.select(['userID','page', 'firstName', 'lastName','ts', 'auth']).show(5, False)

+------+-------------------------+---------+--------+-------------+---------+
|userID|page                     |firstName|lastName|ts           |auth     |
+------+-------------------------+---------+--------+-------------+---------+
|18    |Cancellation Confirmation|Adriel   |Mendoza |1538943990000|Cancelled|
|32    |Cancellation Confirmation|Diego    |Mckee   |1539033046000|Cancelled|
|125   |Cancellation Confirmation|Mason    |Hart    |1539318918000|Cancelled|
|105   |Cancellation Confirmation|Alexander|Garcia  |1539375441000|Cancelled|
|17    |Cancellation Confirmation|Kayla    |Johnson |1539465584000|Cancelled|
+------+-------------------------+---------+--------+-------------+---------+
only showing top 5 rows



In [30]:
user_ids_who_churn = [user_ids['userID'] for user_ids in churn_logs.select('userID').distinct().collect()]

In [31]:
user_ids_who_downgraded = [user_ids['userID'] for user_ids in downgrade_logs.select('userID').distinct().collect()]

In [32]:
len(user_ids_who_churn)

52

In [33]:
len(user_ids_who_downgraded)

49

In [34]:
user_ids_who_did_both = [x for x in user_ids_who_churn if x in user_ids_who_downgraded]

In [35]:
user_ids_who_did_both

['54', '100025', '100009', '100015', '103', '100012', '200020', '200011', '12']

In [36]:
user_ids_all = [user_ids['userId'] for user_ids in user_log_valid.select('userId').distinct().collect()]

In [37]:
user_ids_who_didnt_churn = [user_ids for user_ids in user_ids_all if user_ids not in user_ids_who_churn]

In [38]:
len(user_ids_who_didnt_churn)

173

In [39]:
#total number of unique user ID
user_log_valid.select('userId').distinct().count()

225

In [40]:
52+173

225

In [41]:
windowval = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)

In [42]:
user_log_valid = user_log_valid.withColumn("phase", Fsum("downgrade").over(windowval))

In [43]:
user_log_valid.filter

<bound method DataFrame.filter of DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: bigint, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string, downgrade: int, churn: int, phase: bigint]>

In [57]:
user_log_valid \
.select(["userId", "ts", "level", "churn", "downgrade", "phase"]) \
.filter(user_log_valid.userId == "54") \
.sort("ts") \
.show(100000,False)

+------+-------------+-----+-----+---------+-----+
|userId|ts           |level|churn|downgrade|phase|
+------+-------------+-----+-----+---------+-----+
|54    |1538353930000|paid |0    |0        |1    |
|54    |1538354180000|paid |0    |0        |1    |
|54    |1538354396000|paid |0    |0        |1    |
|54    |1538354739000|paid |0    |0        |1    |
|54    |1538354749000|paid |0    |0        |1    |
|54    |1538354985000|paid |0    |0        |1    |
|54    |1538355254000|paid |0    |0        |1    |
|54    |1538355255000|paid |0    |0        |1    |
|54    |1538395957000|paid |0    |0        |1    |
|54    |1538396264000|paid |0    |0        |1    |
|54    |1538396638000|paid |0    |0        |1    |
|54    |1538396906000|paid |0    |0        |1    |
|54    |1538397069000|paid |0    |0        |1    |
|54    |1538397315000|paid |0    |0        |1    |
|54    |1538397475000|paid |0    |0        |1    |
|54    |1538397657000|paid |0    |0        |1    |
|54    |1538398061000|paid |0  

In [45]:
get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).hour)

In [46]:
user_log_valid = user_log_valid.withColumn("hour", get_hour(user_log_valid.ts))

In [47]:
user_log_valid.head()

Row(artist=None, auth='Logged In', firstName='Darianna', gender='F', itemInSession=34, lastName='Carpenter', length=None, level='free', location='Bridgeport-Stamford-Norwalk, CT', method='PUT', page='Logout', registration=1538016340000, sessionId=187, song=None, status=307, ts=1542823952000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"', userId='100010', downgrade=0, churn=0, phase=0, hour='18')

In [48]:
#let's look at the useragent from user who churn versus user who doesn't

In [49]:
user_agent_churn = user_log_valid \
.select("userId","userAgent") \
.where(user_log_valid.userId.isin(user_ids_who_churn)) \
.dropDuplicates() \
.groupby("userAgent") \
.agg({"userAgent":"count"}) \
.orderBy(desc("count(userAgent)")) \
.withColumnRenamed("count(userAgent)", "userAgent_churn")

#user_log.select("artist").groupBy("artist").agg({"artist":"count"}).orderBy(desc("count(Artist)")).show(1)

#user_ids_who_churn

In [50]:
user_agent_not_churn = user_log_valid \
.select("userId","userAgent") \
.where(user_log_valid.userId.isin(user_ids_who_didnt_churn)) \
.dropDuplicates() \
.groupby("userAgent") \
.agg({"userAgent":"count"}) \
.orderBy(desc("count(userAgent)")) \
.withColumnRenamed("count(userAgent)", "userAgent_not_churn")


In [51]:
data = user_agent_churn.join(user_agent_not_churn,'userAgent','outer').fillna(0)

In [52]:
data.show(data.count())

+--------------------+---------------+-------------------+
|           userAgent|userAgent_churn|userAgent_not_churn|
+--------------------+---------------+-------------------+
|"Mozilla/5.0 (Mac...|              0|                  1|
|"Mozilla/5.0 (Win...|              0|                  4|
|Mozilla/5.0 (X11;...|              1|                  0|
|"Mozilla/5.0 (Mac...|              5|                 11|
|"Mozilla/5.0 (Mac...|              1|                  1|
|Mozilla/5.0 (Maci...|              1|                  1|
|Mozilla/5.0 (Wind...|              0|                  1|
|Mozilla/5.0 (Wind...|              1|                  3|
|Mozilla/5.0 (comp...|              0|                  3|
|"Mozilla/5.0 (Win...|              2|                  5|
|Mozilla/5.0 (Maci...|              2|                  7|
|"Mozilla/5.0 (Win...|              1|                  0|
|"Mozilla/5.0 (iPh...|              0|                  1|
|"Mozilla/5.0 (Win...|              1|                  

In [53]:
#user_log.select("artist").groupBy("artist").agg({"artist":"count"}).orderBy(desc("count(Artist)")).show(1)

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.